<a href="https://colab.research.google.com/github/andrew11morozovtwo/telegrambot1/blob/main/bot_safety_rules_always_bot_08_01_2025_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
# pip install pyTelegramBotAPI

# pip install openai
import telebot
import logging
from openai import OpenAI

import csv
from datetime import datetime
import os

# Настройка логирования
logging.basicConfig(level=logging.INFO)

# Замените 'YOUR_API_TOKEN' на ваш токен Telegram
API_TOKEN = 'YOUR_API_TOKEN'
bot = telebot.TeleBot(API_TOKEN)

# Настройка OpenAI
client = OpenAI(
    api_key="_",
    base_url="https://api.proxyapi.ru/openai/v1",
)

# Путь к файлу на Google Диске
file_path = '/content/gdrive/MyDrive/telegram_bot_logs.csv'

# Словарь для хранения истории разговора
conversation_history = {}

# Команда /start
@bot.message_handler(commands=['start'])
def send_welcome(message):
    chat_id = message.chat.id
    # Инициализация истории разговора для нового чата
    if chat_id not in conversation_history:
        conversation_history[chat_id] = []
    bot.reply_to(message, "Добро пожаловать в канал 'Безопасность всегда'! Как я могу помочь? Бот версии 05_01_2025 г")

# Обработка текстовых сообщений
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    chat_id = message.chat.id
    user_message = message.text
    message_type = 'text'
    process_message(message, user_message, message_type, chat_id)

# Обработка сообщений с фотографиями
@bot.message_handler(content_types=['photo'])
def handle_photo_message(message):
    chat_id = message.chat.id
    user_message = message.caption if message.caption else "Фото без подписи"
    message_type = 'photo'
    process_message(message, user_message, message_type, chat_id)

# Обработка сообщений с видео
@bot.message_handler(content_types=['video'])
def handle_video_message(message):
    chat_id = message.chat.id
    user_message = message.caption if message.caption else "Видео без подписи"
    message_type = 'video'
    process_message(message, user_message, message_type, chat_id)

# Обработка голосовых сообщений
@bot.message_handler(content_types=['voice'])
def handle_voice_message(message):
    chat_id = message.chat.id
    user_message = "Голосовое сообщение без текста"
    message_type = 'voice'
    process_message(message, user_message, message_type, chat_id)

# Обработка аудио сообщений
@bot.message_handler(content_types=['audio'])
def handle_audio_message(message):
    chat_id = message.chat.id
    user_message = "Аудио сообщение без текста"
    message_type = 'audio'
    process_message(message, user_message, message_type, chat_id)

# Обработка опросов
@bot.message_handler(content_types=['poll'])
def handle_poll_message(message):
    chat_id = message.chat.id
    user_message = f"Опрос: {message.poll.question}"
    message_type = 'poll'
    process_message(message, user_message, message_type, chat_id)

# Функция для инициализации файла (создаем файл и пишем заголовки, если он не существует)
def initialize_log_file():
    if not os.path.exists(file_path):
        with open(file_path, 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['chat_id', 'datetime', 'message', 'user_message', 'message_type', 'ai_response'])

# Функция для записи данных в файл
def log_to_file(chat_id, user_message, message_type, ai_response):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Текущее время
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([chat_id, current_time, user_message, user_message, message_type, ai_response])


# Общая функция для обработки сообщений
def process_message(message, user_message, message_type, chat_id):
    # Проверяем, существует ли история для данного chat_id
    if chat_id not in conversation_history:
        conversation_history[chat_id] = []

    # Добавляем сообщение пользователя в историю разговора
    conversation_history[chat_id].append({"role": "user", "content": user_message})
    logging.info(f"Получено сообщение от пользователя: {user_message} (Тип: {message_type})")

    # Запрос к OpenAI с историей разговора
    try:
        chat_completion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=conversation_history[chat_id] + [
                {"role": "system", "content": "Вы бот-администратор в телеграмм-канале 'Безопасность всегда'. Ваша задача — обеспечивать безопасность подписчиков через информирование о правилах поведения. Каждый день мы публикуем важные напоминания. Комментируя посты, общайтесь с читателями от мужского лица на 'Вы', учитывая контекст каждого поста и комментарии пользователей. Включайте эмодзи в ваши ответы, чтобы сделать их более привлекательными и выразительными, добавляя полезные советы и интересные факты."},
                {"role": "user", "content": f"В телеграмм-канале 'Безопасность всегда' опубликован пост: '{user_message}' (Тип: {message_type}). Напиши увлекательный комментарий к этому посту, подчеркивающий важность правил безопасности. Включи три дополнительных правила безопасности по тематике поста, разделенных на абзацы. Используй много эмодзи для привлечения внимания!"}
            ]
        )
        # Получаем ответ от AI
        ai_response = chat_completion.choices[0].message.content
        bot.reply_to(message, ai_response)

        # Логирование данных в файл
        log_to_file(chat_id, user_message, message_type, ai_response)

        # Добавляем ответ AI в историю разговора
        conversation_history[chat_id].append({"role": "assistant", "content": ai_response})
    except Exception as e:
        logging.error(f"Ошибка при обращении к OpenAI: {e}")
        bot.reply_to(message, "Извините, произошла ошибка при обработке вашего запроса.")

# Запуск бота
if __name__ == '__main__':
    initialize_log_file()  # Инициализация файла логов
    bot.polling(none_stop=True)

